# Curved Barcode OCR Pipeline

**Goal:** Detect a barcode on a curved surface using `Piero2411/YOLOV8s-Barcode-Detection`,
then attempt to decode the text via two complementary paths:

1. **Direct decode** — `zxingcpp` (structured barcode symbology, pure-Python, no system libs needed)
2. **OCR fallback** — cylindrical unwrap → `EasyOCR` (when curvature distorts the symbology)

```
curved_barcode.jpg
       │
       ▼
  YOLOv8s detect          ← Piero2411/YOLOV8s-Barcode-Detection
       │
       ▼
  Crop ROI + preprocess
       │
   ┌───┴────────────────┐
   ▼                    ▼
zxingcpp decode   Cylindrical unwrap
   │                    │
   ▼                    ▼
barcode string     EasyOCR text
```

## 1  Install dependencies

In [ ]:
# One-time install — skip if you already have these
import subprocess, sys

packages = [
    "ultralytics",        # YOLOv8
    "huggingface_hub",    # download model weights
    "opencv-python",      # image processing
    "Pillow",
    "numpy",
    "matplotlib",
    "scipy",
    "scikit-image",       # polar / cylindrical transforms
    "zxingcpp",           # structured barcode decode (pure-Python, no system libs)
    "easyocr",            # OCR fallback for curved text
]

subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", *packages])
print("All packages installed.")

## 2  Imports & config

In [ ]:
from pathlib import Path
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from PIL import Image
from huggingface_hub import hf_hub_download
from ultralytics import YOLO
import zxingcpp
import easyocr

IMAGE_PATH = Path("curved_barcode.jpg")   # relative to notebook location
HF_REPO    = "Piero2411/YOLOV8s-Barcode-Detection"
MODEL_FILE = "YOLOV8s_Barcode_Detection.pt"

print(f"Image exists: {IMAGE_PATH.exists()}")

## 3  Load & inspect the source image

In [ ]:
img_bgr = cv2.imread(str(IMAGE_PATH))
img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
h, w = img_bgr.shape[:2]
print(f"Image size: {w} × {h} px  (width × height)")

plt.figure(figsize=(4, 6))
plt.imshow(img_rgb)
plt.title("Source: curved_barcode.jpg")
plt.axis("off")
plt.tight_layout()
plt.show()

## 4  Download the YOLO model from HuggingFace

In [ ]:
model_path = hf_hub_download(repo_id=HF_REPO, filename=MODEL_FILE)
print(f"Model cached at: {model_path}")

yolo = YOLO(model_path)
print("Model loaded.")

## 5  Run YOLO barcode detection

In [ ]:
results = yolo.predict(
    source=str(IMAGE_PATH),
    imgsz=640,
    conf=0.25,      # lower threshold — curved barcodes can look unusual
    iou=0.45,
    verbose=False,
)

result = results[0]
boxes  = result.boxes

print(f"Detections: {len(boxes)}")
for i, box in enumerate(boxes):
    x1, y1, x2, y2 = [int(v) for v in box.xyxy[0]]
    conf  = float(box.conf[0])
    cls   = int(box.cls[0])
    label = result.names[cls]
    print(f"  [{i}] class={label}  conf={conf:.2f}  bbox=({x1},{y1})-({x2},{y2})")

In [ ]:
# Visualise detections
annotated = result.plot()                          # returns BGR numpy array
annotated_rgb = cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(4, 6))
plt.imshow(annotated_rgb)
plt.title("YOLO detections")
plt.axis("off")
plt.tight_layout()
plt.show()

## 6  Crop the best detection (or full image as fallback)

If YOLO finds a box we crop to it; otherwise we treat the whole image as the ROI.

In [ ]:
PAD = 10   # pixel padding around the crop

if len(boxes) > 0:
    # pick the detection with highest confidence
    best_idx = int(boxes.conf.argmax())
    x1, y1, x2, y2 = [int(v) for v in boxes[best_idx].xyxy[0]]
    x1, y1 = max(0, x1 - PAD), max(0, y1 - PAD)
    x2, y2 = min(w, x2 + PAD), min(h, y2 + PAD)
    roi_bgr = img_bgr[y1:y2, x1:x2]
    print(f"Cropped to detection [{best_idx}]: ({x1},{y1}) → ({x2},{y2})")
else:
    roi_bgr = img_bgr.copy()
    print("No detection — using full image as ROI.")

roi_rgb = cv2.cvtColor(roi_bgr, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(3, 5))
plt.imshow(roi_rgb)
plt.title("ROI (cropped barcode)")
plt.axis("off")
plt.tight_layout()
plt.show()

## 7  Path A — zxingcpp direct decode

`zxingcpp` understands structured barcode symbologies (EAN, Code128, QR, DataMatrix, …).
It is a pure-Python binding to the ZXing-C++ library — no `libzbar` system package needed.
On a curved image it often fails — but it is fast, so we always try it first.

In [ ]:
def try_zxing(img_bgr: np.ndarray) -> list[dict]:
    """Attempt zxingcpp decode on a BGR image; return list of result dicts."""
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    results = zxingcpp.read_barcodes(img_rgb)
    return [
        {"type": str(r.format), "data": r.text, "position": r.position}
        for r in results
        if r.valid
    ]

zxing_results = try_zxing(roi_bgr)

if zxing_results:
    print("zxingcpp decoded successfully:")
    for r in zxing_results:
        print(f"  type={r['type']}  data='{r['data']}'")
else:
    print("zxingcpp: no decode — curvature likely distorted the symbology. Continuing to OCR path.")

## 8  Path B — Cylindrical / polar unwrap

A barcode photographed on a curved surface (e.g. a can) follows a roughly
**cylindrical projection**.  The key distortion is that the left/right columns
of the barcode are "compressed" compared to the centre column.

**Strategy:**
1. Convert the ROI to grayscale and upscale for better resolution.
2. Estimate the curvature by fitting a polynomial to the horizontal intensity
   profile of the barcode lines.
3. Apply a row-wise horizontal stretch (inverse cylindrical warp) to flatten it.
4. Feed the rectified strip to EasyOCR.

In [ ]:
from scipy.ndimage import map_coordinates


def preprocess_roi(img_bgr: np.ndarray, scale: float = 2.0) -> np.ndarray:
    """Grayscale + upscale + CLAHE contrast enhancement."""
    gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
    if scale != 1.0:
        nw = int(gray.shape[1] * scale)
        nh = int(gray.shape[0] * scale)
        gray = cv2.resize(gray, (nw, nh), interpolation=cv2.INTER_CUBIC)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    return clahe.apply(gray)


def cylindrical_unwarp(gray: np.ndarray, strength: float = 0.4) -> np.ndarray:
    """
    Inverse cylindrical warp.

    Each row is treated as a horizontal cross-section of the cylinder.
    Columns near the edges of the image are "stretched" to compensate for the
    foreshortening caused by curvature.

    strength ∈ [0, 1] — 0 = no correction, 1 = full hemisphere correction.
    """
    h, w = gray.shape
    cx = w / 2.0

    # Map each output column x_out → source column x_src
    # using the inverse of: x_out = cx + R * sin(θ),  x_src = cx + R * θ
    # Approximation: x_src ≈ cx + (x_out - cx) * (1 - strength * ((x_out-cx)/cx)^2)
    x_out = np.arange(w, dtype=np.float32)
    norm  = (x_out - cx) / cx                          # [-1, 1]
    x_src = cx + (x_out - cx) * (1.0 + strength * norm ** 2)
    x_src = np.clip(x_src, 0, w - 1)

    row_idx = np.tile(np.arange(h, dtype=np.float32).reshape(-1, 1), (1, w))
    col_idx = np.tile(x_src, (h, 1))

    unwarped = map_coordinates(gray, [row_idx, col_idx], order=1, mode="nearest")
    return unwarped.astype(np.uint8)


# --- run it ---
gray_roi   = preprocess_roi(roi_bgr, scale=2.0)
unwarped   = cylindrical_unwarp(gray_roi, strength=0.4)

fig, axes = plt.subplots(1, 2, figsize=(10, 4))
axes[0].imshow(gray_roi,  cmap="gray"); axes[0].set_title("Preprocessed ROI");  axes[0].axis("off")
axes[1].imshow(unwarped,  cmap="gray"); axes[1].set_title("After cylindrical unwarp"); axes[1].axis("off")
plt.tight_layout()
plt.show()

### 8b  Re-try zxingcpp on the unwarped image

In [ ]:
unwarped_bgr = cv2.cvtColor(unwarped, cv2.COLOR_GRAY2BGR)
zxing_unwarp_results = try_zxing(unwarped_bgr)

if zxing_unwarp_results:
    print("zxingcpp decoded the UNWARPED image:")
    for r in zxing_unwarp_results:
        print(f"  type={r['type']}  data='{r['data']}'")
else:
    print("zxingcpp still could not decode — proceeding to EasyOCR.")

## 9  Path B (continued) — EasyOCR on the unwarped strip

In [ ]:
# Initialise EasyOCR reader (downloads models on first run, ~100 MB)
reader = easyocr.Reader(["en"], gpu=False)
print("EasyOCR reader ready.")

In [ ]:
def run_easyocr(gray_img: np.ndarray, reader) -> list[dict]:
    """Run EasyOCR and return list of {text, conf, bbox} dicts."""
    ocr_results = reader.readtext(gray_img, detail=1, paragraph=False)
    return [
        {"text": text, "conf": conf, "bbox": bbox}
        for bbox, text, conf in ocr_results
    ]


ocr_on_unwarped = run_easyocr(unwarped, reader)

print(f"EasyOCR found {len(ocr_on_unwarped)} text region(s) on unwarped image:")
for r in ocr_on_unwarped:
    print(f"  conf={r['conf']:.2f}  text='{r['text']}'")

In [ ]:
# Also try directly on the original (non-unwarped) grayscale for comparison
gray_direct = preprocess_roi(roi_bgr, scale=2.0)
ocr_on_direct = run_easyocr(gray_direct, reader)

print(f"EasyOCR found {len(ocr_on_direct)} text region(s) on preprocessed-only image:")
for r in ocr_on_direct:
    print(f"  conf={r['conf']:.2f}  text='{r['text']}'")

## 10  Visualise OCR results on the unwarped image

In [ ]:
def draw_ocr_results(gray_img: np.ndarray, ocr_results: list[dict]) -> np.ndarray:
    """Draw bounding boxes and labels onto a copy of the image."""
    vis = cv2.cvtColor(gray_img, cv2.COLOR_GRAY2RGB)
    for r in ocr_results:
        pts = np.array(r["bbox"], dtype=np.int32)
        cv2.polylines(vis, [pts], isClosed=True, color=(255, 80, 0), thickness=2)
        org = tuple(pts[0])
        cv2.putText(vis, r["text"], org, cv2.FONT_HERSHEY_SIMPLEX,
                    0.6, (255, 80, 0), 2, cv2.LINE_AA)
    return vis


vis_unwarped = draw_ocr_results(unwarped, ocr_on_unwarped)

fig, axes = plt.subplots(1, 2, figsize=(12, 5))
axes[0].imshow(unwarped, cmap="gray")
axes[0].set_title("Unwarped (input to EasyOCR)")
axes[0].axis("off")
axes[1].imshow(vis_unwarped)
axes[1].set_title("EasyOCR detections")
axes[1].axis("off")
plt.tight_layout()
plt.show()

## 11  Aggregate & rank results

In [ ]:
print("═" * 55)
print(" FINAL RESULTS SUMMARY")
print("═" * 55)

all_candidates = []

for r in zxing_results:
    all_candidates.append({"source": "zxingcpp (original)",  "text": r["data"], "conf": 1.0, "type": r["type"]})

for r in zxing_unwarp_results:
    all_candidates.append({"source": "zxingcpp (unwarped)",  "text": r["data"], "conf": 1.0, "type": r["type"]})

for r in ocr_on_unwarped:
    all_candidates.append({"source": "EasyOCR (unwarped)",   "text": r["text"], "conf": r["conf"], "type": "OCR"})

for r in ocr_on_direct:
    all_candidates.append({"source": "EasyOCR (direct)",     "text": r["text"], "conf": r["conf"], "type": "OCR"})

if all_candidates:
    all_candidates.sort(key=lambda x: x["conf"], reverse=True)
    print(f"{'Source':<28} {'Type':<10} {'Conf':>5}  Text")
    print("-" * 55)
    for c in all_candidates:
        print(f"{c['source']:<28} {c['type']:<10} {c['conf']:>5.2f}  {c['text']}")
    print()
    best = all_candidates[0]
    print(f"Best candidate  →  '{best['text']}'  (source: {best['source']}, conf={best['conf']:.2f})")
else:
    print("No text could be extracted.")
    print("Possible next steps:")
    print("  • Increase image resolution / lighting")
    print("  • Tune cylindrical_unwarp strength parameter")
    print("  • Try a dedicated curved-text detector (e.g. ABCNet, TextBPN)")

print("═" * 55)

## 12  Tuning guide

| Parameter | Where | Effect |
|---|---|---|
| `conf=` in `yolo.predict` | §5 | Lower → more detections, higher false-positive rate |
| `scale=` in `preprocess_roi` | §8 | Higher → finer OCR, slower |
| `strength=` in `cylindrical_unwarp` | §8 | Match to the curvature radius of your container |
| `gpu=False` in `easyocr.Reader` | §9 | Switch to `True` if a CUDA GPU is available |

### Possible upgrades for production

* **Better rectification** — fit an actual cylinder model using the vanishing lines of the barcode bars (RANSAC + homography).
* **Curved-text OCR models** — ABCNet v2, TextBPN, or TESTR are specifically designed for curved/arbitrary-shape text.
* **Richer symbology support** — `zxingcpp` already covers EAN, Code128, QR, DataMatrix, PDF417, and more out of the box.
* **Data augmentation** — fine-tune the YOLO model with more examples of wrapped/curved barcodes if detection confidence is low.